In [1]:
using QuadGK
using DelimitedFiles
using Trapz
using Plots
using Interpolations
using PhysicalConstants.CODATA2018: c_0
using SpecialFunctions: besselj1

In [2]:
# Define cosmological parameters for DUSTGRAIN-pathfinder simulations
# as they appear in https://doi:10.1093/mnras/sty2465 for LCDM background
Ω_b = 0.0481
h = 0.6731
H0 = h*100 # Hubble constant in km/s/Mpc

67.31

In [3]:
# Define input folders
inpath = "/home/alessandro/phd/scripts/Dustgrain_outs/"
pk_path = inpath*"Pknl/"

"/home/alessandro/phd/scripts/Dustgrain_outs/Pknl/"

In [4]:
# Selecting cosmology
cosmo = "lcdm"
method = "HM"

"HM"

In [5]:
# For lcdm
Ω_m = 0.31345
Ω_cdm = Ω_m - Ω_b
Ω_de = 1 - Ω_m

plot_label = "LCDM"

"LCDM"

In [6]:
# Load matter power spectrum from file (assuming it contains only P_kappa(ell) values)
Pknl = readdlm(pk_path*"$cosmo"*"_$method"*".txt",',')
k = readdlm(pk_path*"k_"*"$cosmo"*"_$method"*".txt",',')
z = readdlm(pk_path*"z_"*"$cosmo"*"_$method"*".txt",',')

100×1 Matrix{Float64}:
 0.0
 0.04040404040404041
 0.08080808080808081
 0.12121212121212122
 0.16161616161616163
 0.20202020202020204
 0.24242424242424243
 0.2828282828282829
 0.32323232323232326
 0.36363636363636365
 ⋮
 3.676767676767677
 3.7171717171717176
 3.757575757575758
 3.7979797979797985
 3.8383838383838387
 3.878787878787879
 3.9191919191919196
 3.95959595959596
 4.0

In [7]:
Pknl

100×214 Matrix{Float64}:
 332.158   458.249   632.13    …  7.40729e-5  3.6735e-5   1.82537e-5
 318.293   439.122   605.746      6.6725e-5   3.30669e-5  1.64197e-5
 304.949   420.713   580.352      6.04096e-5  2.99379e-5  1.487e-5
 292.133   403.031   555.961      5.5003e-5   2.72509e-5  1.35275e-5
 279.845   386.078   532.577      5.0345e-5   2.49335e-5  1.23741e-5
 268.082   369.851   510.192   …  4.63436e-5  2.29292e-5  1.1376e-5
 256.838   354.339   488.795      4.27931e-5  2.11859e-5  1.05079e-5
 246.103   339.529   468.365      3.96632e-5  1.96313e-5  9.74025e-6
 235.864   325.403   448.879      3.68947e-5  1.82586e-5  9.05671e-6
 226.106   311.941   430.31       3.44337e-5  1.70386e-5  8.45174e-6
   ⋮                           ⋱                          
  24.2346   33.4356   46.1256     3.13792e-6  1.57632e-6  7.95546e-7
  23.8248   32.8701   45.3456     3.08578e-6  1.55038e-6  7.82513e-7
  23.4251   32.3188   44.585      3.0352e-6   1.52518e-6  7.69859e-7
  23.0354   31.7811   

In [58]:
# Sources redshift
zs = 2.0

2.0

In [59]:
# Speed of light in km/s
# c = c_0/1e3
c = 299792.458

299792.458

In [60]:
# Dimemensionless Hubble parameter
function E(z)
    return sqrt(Ω_m * (1 + z)^3 + Ω_de)
end

# Comoving raidal distance r(z)
function r(z)
    integrand(z_prime) = 1 / E(z_prime)
    result, _ = quadgk(integrand, 0.0, z)
    return (c / H0) * result
end

r (generic function with 1 method)

In [61]:
# Top-hat window function
function W_th(ℓ, θ)
    return 2 * besselj1(ℓ * θ) / (ℓ * θ)
end

W_th (generic function with 1 method)

In [62]:
Pknl = Pknl/h^3
k = k*h

214-element Vector{Float64}:
   2.265318050000001e-5
   3.161506017351792e-5
   4.412237079800598e-5
   6.15777289099528e-5
   8.593864357531304e-5
   0.0001199370387687517
   0.0001673856215337019
   0.00023360545319319213
   0.00032602267304428564
   0.0004550012933603849
   ⋮
  80.40024101172071
 100.40809774488754
 125.39497352100194
 156.5999131293512
 195.570301611914
 244.2385957199252
 305.01815023844637
 380.92289099782715
 475.71676889624354

In [63]:
z = vec(z)
k = vec(k)

214-element Vector{Float64}:
   2.265318050000001e-5
   3.161506017351792e-5
   4.412237079800598e-5
   6.15777289099528e-5
   8.593864357531304e-5
   0.0001199370387687517
   0.0001673856215337019
   0.00023360545319319213
   0.00032602267304428564
   0.0004550012933603849
   ⋮
  80.40024101172071
 100.40809774488754
 125.39497352100194
 156.5999131293512
 195.570301611914
 244.2385957199252
 305.01815023844637
 380.92289099782715
 475.71676889624354

In [64]:
# Pzk = interpolate(Pknl, BSpline(Quadratic(Line(OnCell()))))
Pzk = interpolate((z,k), Pknl, Gridded(Linear()))
# Pzk = CubicSplineInterpolation(zk_nodes, Pknl)

100×214 interpolate((::Vector{Float64},::Vector{Float64}), ::Matrix{Float64}, Gridded(Linear())) with element type Float64:
 11711.9    16157.9   22289.0   30739.3   …  0.00129528   0.000643628
 11223.1    15483.5   21358.7   29456.3      0.00116594   0.000578959
 10752.5    14834.4   20463.3   28221.5      0.00105561   0.000524318
 10300.6    14210.9   19603.3   27035.5      0.00096087   0.000476982
  9867.35   13613.2   18778.7   25898.4      0.000879156  0.000436312
  9452.61   13041.0   17989.4   24809.9   …  0.000808487  0.000401121
  9056.15   12494.0   17235.0   23769.4      0.000747018  0.000370508
  8677.63   11971.8   16514.6   22776.0      0.0006922    0.000343442
  8316.59   11473.7   15827.5   21828.5      0.000643801  0.000319341
  7972.53   10999.1   15172.8   20925.5      0.000600782  0.000298009
     ⋮                                    ⋱               
   854.515   1178.94   1626.39   2243.29     5.55812e-5   2.8051e-5
   840.063   1159.0    1598.89   2205.35     5.46

In [65]:
# Smoothed convergence
function κ_sm(z, θ_sm)
    integrand(ℓ) = Pzk(z, (ℓ+0.5)/r(z)) * W_th(ℓ,θ_sm)^2 * ℓ
    result, _ = quadgk(integrand, 1.0, 10000)
    return 2 * π * result   
end

κ_sm (generic function with 1 method)

In [86]:
κ_sm(5e-3, 10)

305.81150300693673

In [67]:
# Lensing efficiency
function W(z)
    return 1.5 * Ω_m * (H0/c)^2 * (1+z) * r(z) * (1-(r(z)/r(zs)))
end

W (generic function with 1 method)

In [75]:
W(2.6)

-7.590475058759298e-5

In [87]:
# Moments functional
function C(θ_sm, t)
    integrand(z) = W(z)^t * κ_sm(z,θ_sm)^(t-1) / E(z) / r(z)^(2*(t-1))
    result, _ = quadgk(integrand, 5e-3 , zs)
    return (c/H0) * result
end

C (generic function with 1 method)

In [88]:
# Smoothing radius range
θ_s = range(2,stop=20,length=50)
length(θ_s)

50

In [89]:
# Functional for the second κ moment
C_2 = zeros(length(θ_s))

50-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [90]:

for (i,θ) in enumerate(θ_s)
    C_2[i] = C(θ,2)
end

C_2

In [ ]:
# Second moment
Q2 = 1
κ_2 = Q2*C_2